<a href="https://colab.research.google.com/github/Felipe34515/Trabajos-BI/blob/main/LAB1_BI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **LABORATORIO 1 - INTELIGENCIA DE NEGOCIOS**

Santiago Pardo - 202013025



Luis Plazas - 202013155




Felipe Rueda - 202010903

## DESCRIPCIÓN DEL PROBLEMA

### Contexto



En el vasto panorama del mundo empresarial, la comprensión profunda de los clientes es la clave para el éxito sostenible. La segmentación de clientes emerge como una poderosa herramienta estratégica que redefine la forma en que las empresas abordan sus mercados. Este proceso implica la división cuidadosa y deliberada de la base de clientes en grupos homogéneos, con características y comportamientos similares. La segmentación de clientes va más allá de la simple categorización; es la llave maestra que desbloquea el potencial de la personalización. Al centrarse en las similitudes y preferencias compartidas dentro de cada segmento, las empresas pueden anticipar las necesidades de sus clientes, crear mensajes más efectivos y diseñar productos y servicios que resuenen con precisión.



AlpesInsight, es una empresa pionera en la intersección de servicios financieros y tecnología. En su búsqueda constante para ofrecer experiencias personalizadas y seguras a sus clientes, AlpesInsight está expandiendo su equipo con la incorporación de expertos en aprendizaje automático para enfrentar el desafío de aprovechar datos de transacciones con tarjetas de crédito para comprender los patrones de comportamiento y preferencias de los clientes. AlpesInsight se esfuerza por desarrollar soluciones innovadoras que permitan personalizar ofertas financieras y servicios, mejorando así la satisfacción del cliente y fortaleciendo la seguridad de las transacciones. El equipo ha dedicido utilizar técnicas de agrupación para resolver este reto



### Diccionario de los Datos (En Inglés)



| Atributo                         | Significado                                                                                                                   |
| -------------------------------- | ----------------------------------------------------------------------------------------------------------------------------- |
| CUST_ID                          | Credit card holder ID                                                                                                         |
| BALANCE                          | Monthly average balance (based on daily balance averages)                                                                     |
| BALANCE_FREQUENCY                | Ratio of last 12 months with balance. (1: Frequently updated, 0: Not frequently updated)                                      |
| PURCHASES                        | Total purchase amount spent during last 12 months                                                                             |
| ONEOFF_PURCHASES                 | Total amount of one-off purchases                                                                                             |
| INSTALLMENTS_PURCHASES           | Total amount of installment purchases                                                                                         |
| CASH_ADVANCE                     | Total cash-advance amount                                                                                                     |
| PURCHASES_FREQUENCY              | Frequency of purchases (Percent of months with at least one purchase). (1: Frequently purchased, 0: Not frequently purchased) |
| ONEOFF_PURCHASES_FREQUENCY       | Frequency of one-off-purchases. (1: Frequently purchased, 0: Not frequently purchased)                                        |
| PURCHASES_INSTALLMENTS_FREQUENCY | Frequency of installment purchases. (1: Frequently purchased, 0: Not frequently purchased)                                    |
| CASHADVANCE_FREQUENCY            | Cash-Advance frequency                                                                                                        |
| CASH_ADVANCE_TRX                 | Average amount per cash-advance transaction                                                                                   |
| PURCHASES_TRX                    | Average amount per purchase transaction                                                                                       |
| CREDIT_LIMIT                     | Credit limit                                                                                                                  |
| PAYMENTS                         | Total payments (Due amount paid by the customer to decrease their statement balance) in the period                            |
| MINIMUM_PAYMENTS                 | Total minimum payments due in the period                                                                                      |
| PRC_FULL_PAYMEN                  | Percentage of months with full payment of the due statement balance                                                           |
| TENURE                           | Number of months as a customer                                                                                                |

## ENTENDIMIENTO DE LOS DATOS

(Aquí pegas el tablero de control)

## PREPARACIÓN DE DATOS

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('202410_Laboratorio 1 - Agrupación_Customer_Data.csv',sep=',')

In [5]:
df

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,C10001,40.900749,0.818182,95.40,0.00,95.40,0.000000,0.166667,0.000000,0.083333,0.000000,0,2,1000.0,201.802084,139.509787,0.000000,12
1,C10002,3202.467416,0.909091,0.00,0.00,0.00,6442.945483,0.000000,NaN,0.000000,0.250000,4,0,7000.0,4103.032597,1072.340217,0.222222,12
2,C10003,2495.148862,1.000000,773.17,773.17,0.00,0.000000,1.000000,1.000000,0.000000,0.000000,0,12,7500.0,622.066742,627.284787,0.000000,12
3,C10004,1666.670542,0.636364,1499.00,1499.00,0.00,205.788017,0.083333,0.083333,0.000000,0.083333,1,1,7500.0,0.000000,NaN,0.000000,12
4,C10005,817.714335,1.000000,16.00,16.00,0.00,0.000000,0.083333,0.083333,0.000000,0.000000,0,1,1200.0,678.334763,244.791237,0.000000,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8956,C19186,28.493517,1.000000,291.12,0.00,291.12,0.000000,1.000000,0.000000,0.833333,0.000000,0,6,1000.0,325.594462,48.886365,0.500000,6
8957,C19187,19.183215,1.000000,300.00,0.00,300.00,0.000000,1.000000,0.000000,0.833333,0.000000,0,6,1000.0,275.861322,NaN,0.000000,6
8958,C19188,23.398673,0.833333,144.40,0.00,144.40,0.000000,0.833333,0.000000,0.666667,0.000000,0,5,1000.0,81.270775,82.418369,0.250000,6
8959,C19189,13.457564,0.833333,0.00,0.00,0.00,36.558778,0.000000,0.000000,0.000000,0.166667,2,0,500.0,52.549959,55.755628,0.250000,6


Eliminar datos nulos

In [9]:
df.shape

(8961, 18)

In [10]:
df = df.dropna()

In [11]:
df.shape

(8619, 18)

## MODELAMIENTO

## VALIDACIÓN